In [ ]:
from typing import Tuple, List

import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
FIXED_SCENARIO_RETURN = .07

# Load data

In [ ]:
def get_avg_return_scenario(data: pd.DataFrame, avg_return: float, data_key: str = "Depotwert") -> \
        List[float]:
    days_list = [(data["Datum"].iloc[i] - data["Datum"].iloc[0]).days for i in
                 range(len(data["Datum"]))]
    return [data[data_key].iloc[0] * (1 + avg_return) ** (
        (i / 365)) for i in days_list]

In [ ]:
def load_data(filename: str) -> Tuple[pd.DataFrame, float]:
    """Load data from csv file and return it as a pandas dataframe"""
    df = pd.read_csv(filename, sep=";", decimal=",", parse_dates=["Datum"],
                     date_format="%d.%m.%Y")
    avg_return = (df["Depotwert"].iloc[-1] / df["Depotwert"].iloc[0]) ** (
            365 / (df["Datum"].iloc[-1] - df["Datum"].iloc[0]).days) - 1
    df["Avg scenario"] = get_avg_return_scenario(df, avg_return)
    df["Fixed scenario"] = get_avg_return_scenario(df, FIXED_SCENARIO_RETURN)
    return df, avg_return

In [ ]:
data, avg_return = load_data("~/Desktop/Depotwerte.csv")
data.head()

# Fortune history line plot

Using just a line diagram.

In [ ]:
fig = go.Figure()
delta_depot_value = data["Depotwert"].diff()
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Depotwert"], name="Depotwert"))
fig.add_trace(go.Scatter(x=data["Datum"], y=delta_depot_value, name="Depotschwankung"))
fig.add_trace(
    go.Scatter(x=data["Datum"], y=data["Avg scenario"],
               name=f"Avg Scenario ({round(avg_return * 100, 2)}%)"))
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Fixed scenario"],
                         name=f"Fixed Scenario ({round(FIXED_SCENARIO_RETURN * 100, 2)}%)"))
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Net Worth"], name="Net Worth"))
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Davon nicht Depot"], name="Davon nicht Depot"))
pass

In [ ]:
fig.show()

# Fortune proposition history

Using a stacked area chart.

In [ ]:
fig = px.area(data, x="Datum", y=["Depotwert", "Davon nicht Depot"], title="Depotwert")
fig.add_trace(go.Scatter(x=data["Datum"], y=data["Net Worth"], name="Net Worth",
                         line=go.scatter.Line(color="limegreen", dash="dash")))
fig.show()

# Current fortune proposition (pie)

In [ ]:
labels = ["Depotwert", "Davon nicht Depot"]
values = [data["Depotwert"].iloc[-1], data["Davon nicht Depot"].iloc[-1]]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

# Depot value fluctuation histogram

In [ ]:
delta_depot_value = delta_depot_value[1:]
stddev = delta_depot_value.std()
median = delta_depot_value.median()
max_count = delta_depot_value.value_counts().max()

In [ ]:
def calculate_share_of_values_being_within_stddev(k) -> float:
    return len(
        [i for i in delta_depot_value if median - k * stddev <= i <= median + k * stddev]) / len(
        delta_depot_value)

In [ ]:
depot_value_fluctuations = sorted(delta_depot_value)
fig = px.histogram(depot_value_fluctuations, marginal="box")
colors = ["red", "blue", "green", "orange", "purple"]
fig.add_vline(x=median, line_width=3, line_dash="dash", line_color=colors[0])
for k in range(1, 3):
    fig.add_vline(x=median + k * stddev, line_width=3, line_dash="dash", line_color=colors[k])
    fig.add_annotation(x=median + k * stddev,
                       text=f"{round(calculate_share_of_values_being_within_stddev(k) * 100, 2)}%",
                       showarrow=False)
    fig.add_vline(x=median - k * stddev, line_width=3, line_dash="dash", line_color=colors[k])
    fig.add_annotation(x=median - k * stddev,
                       text=f"{round(calculate_share_of_values_being_within_stddev(k) * 100, 2)}%",
                       showarrow=False)
fig.show()

# Net worth bubble chart

In [ ]:
sizes = data["Depotwert"]
sizes_normalized = (sizes - sizes.min()) / (sizes.max() - sizes.min())
bubble_chart = go.Scatter(
    x=data["Datum"],
    y=data["Net Worth"],
    mode='markers',
    marker=dict(
        size=sizes_normalized,
        sizemode='diameter',
        sizeref=sizes_normalized.max() / 30,
        sizemin=3,
        color=data["Depotwert"],
        colorscale='Viridis',
        showscale=True
    ),
)
fig = go.Figure(data=[bubble_chart])
fig.update_layout(
    title='Net Worth',
    xaxis_title='Datum',
    yaxis_title='Net Worth',
)
fig.show()